<a href="https://colab.research.google.com/github/milo0914/Fast-whisper/blob/main/de_ill2_Fast_whisper_flash_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#@title 🚀 啟動即時翻譯應用程式 (完整版)
#@markdown ---
#@markdown ### ▶️ 請直接執行此儲存格以啟動所有服務。
#@markdown 執行後，請耐心等待約 1-2 分鐘，直到看見公開的 ngrok URL 輸出。
#@markdown
#@markdown **事前準備提醒：**
#@markdown 1.  **GPU 已啟用**: 請在「執行階段」->「變更執行階段類型」中確認已選取 `T4 GPU`。
#@markdown 2.  **密鑰已設定**: 請在左側「🔑」分頁中設定好 `NGROK_AUTH_TOKEN` 和 `GEMINI_API_KEY`。
#@markdown 3.  **圖示已上傳**: 請在左側「📁」分頁中上傳 `icon-192.png` 和 `icon-512.png`。
#@markdown ---

# STEP 0: 安裝所有必要的依賴項
print("🔄 [1/5] 正在安裝所有依賴項...")
!pip install -q Flask Flask-Cors pyngrok pydub google-generativeai faster-whisper
!apt-get -qq update && apt-get -qq install -y ffmpeg
print("✅ [1/5] 所有依賴項安裝完成。")
print("-" * 50)

# STEP 1: 導入函式庫並設定全域變數
import os
import base64
import tempfile
import shutil
from flask import Flask, jsonify, request, send_from_directory
from flask_cors import CORS
from pyngrok import ngrok
import threading
import time
import io
from pydub import AudioSegment
import google.generativeai as genai
from faster_whisper import WhisperModel
from google.colab import userdata
import traceback # Import traceback module

# STEP 2: 預先加載 AI 模型
print("🔄 [2/5] 正在加載 AI 模型...")
try:
    print("  - 正在加載 Faster-Whisper 模型 (medium)...")
    whisper_model = WhisperModel("medium", device="cuda", compute_type="float16")
    print("  ✅ Whisper 模型已成功加載到 CUDA。")
except Exception as e:
    print(f"  ❌ Whisper 模型加載錯誤: {e}")
    print("     這可能是因為 GPU 記憶體不足或未啟用 GPU。請檢查執行階段類型。")
    raise e

# Gemini 模型將在主執行緒中根據 Secret Key 初始化
gemini_model = None
print("  - Gemini 模型將在稍後初始化...")
print("✅ [2/5] AI 模型準備就緒。")
print("-" * 50)


# STEP 3: 準備前端檔案
print("🔄 [3/5] 正在建立前端應用程式...")
frontend_dir = "/content/static"
os.makedirs(frontend_dir, exist_ok=True)

# 移動上傳的圖示
try:
    shutil.move("/content/icon-192.png", os.path.join(frontend_dir, "icon-192.png"))
    shutil.move("/content/icon-512.png", os.path.join(frontend_dir, "icon-512.png"))
    print("  ✅ 圖示檔案已成功移動。")
except FileNotFoundError:
    print("  ⚠️ 警告: 找不到圖示檔案。應用程式仍可運行，但可能無法正確顯示圖示或作為 PWA 安裝。")
except Exception as e:
    print(f"  ❌ 移動圖示時發生錯誤: {e}")

# --- 建立所有前端檔案 (完整內容) ---
index_html_content = """
<!DOCTYPE html>
<html lang="zh-TW">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>即時日中翻譯 (Fast-Whisper + Gemini)</title>
    <link rel="stylesheet" href="/static/styles.css">
    <link rel="manifest" href="/static/manifest.json">
    <link rel="apple-touch-icon" href="/static/icon-192.png">
</head>
<body>
    <div class="container">
        <header>
            <h1>即時日中翻譯</h1>
            <p>日語 → 中文 (Fast-Whisper + Gemini)</p>
        </header>
        <main>
            <div class="controls">
                <button id="record-btn" class="record-btn">
                    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" fill="currentColor">
                        <path d="M12 14c1.66 0 3-1.34 3-3V5c0-1.66-1.34-3-3-3S9 3.34 9 5v6c0 1.66 1.34 3 3 3z"/>
                        <path d="M17 11h-1c0 2.76-2.24 5-5 5s-5-2.24-5-5H5c0 3.53 2.61 6.43 6 6.92V21h2v-3.08c3.39-.49 6-3.39 6-6.92z"/>
                    </svg>
                </button>
                <p id="status" class="status">點擊按鈕開始錄音</p>
            </div>
            <div class="results">
                <div class="result-box">
                    <h2>轉錄與翻譯</h2>
                    <div id="translation-output" class="text-content"></div>
                </div>
            </div>
            <div class="history">
                <h2>歷史記錄</h2>
                <div id="history-list" class="history-list"></div>
                <button id="clear-history-btn" class="btn">清除歷史記錄</button>
            </div>
        </main>
        <footer>
            <p>由 Faster-Whisper & Gemini AI 提供技術支援</p>
        </footer>
    </div>
    <script src="/static/config.js"></script>
    <script src="/static/app.js"></script>
</body>
</html>
"""
styles_css_content = """
body {
    font-family: "Arial", sans-serif; margin: 0; padding: 0;
    background-color: #f4f7f6; color: #333; display: flex;
    justify-content: center; align-items: center; min-height: 100vh; flex-direction: column;
}
.container {
    background-color: #fff; border-radius: 12px; box-shadow: 0 6px 20px rgba(0, 0, 0, 0.1);
    padding: 30px; width: 90%; max-width: 600px; text-align: center; margin: 20px;
}
header h1 { color: #4a90e2; margin-bottom: 5px; }
header p { color: #666; font-size: 1.1em; margin-top: 0; }
.controls { margin: 30px 0; }
.record-btn {
    background: linear-gradient(45deg, #4a90e2, #7b4ae2); border: none; border-radius: 50%;
    width: 80px; height: 80px; display: flex; justify-content: center; align-items: center;
    cursor: pointer; outline: none; box-shadow: 0 4px 15px rgba(0, 0, 0, 0.2);
    transition: transform 0.2s ease-in-out, box-shadow 0.2s ease-in-out;
}
.record-btn:hover { transform: scale(1.05); box-shadow: 0 6px 20px rgba(0, 0, 0, 0.3); }
.record-btn svg { fill: #fff; width: 40px; height: 40px; }
.record-btn.recording { animation: pulse 1.5s infinite; }
@keyframes pulse {
    0% { box-shadow: 0 0 0 0 rgba(74, 144, 226, 0.7); }
    70% { box-shadow: 0 0 0 20px rgba(74, 144, 226, 0); }
    100% { box-shadow: 0 0 0 0 rgba(74, 144, 226, 0); }
}
.status { margin-top: 20px; font-size: 1.1em; color: #555; }
.results { display: flex; flex-direction: column; gap: 20px; margin-top: 30px; }
.result-box {
    background-color: #f9f9f9; border: 1px solid #eee; border-radius: 8px;
    padding: 20px; text-align: left; min-height: 80px; position: relative;
}
.result-box h2 { color: #4a90e2; font-size: 1.2em; margin-top: 0; margin-bottom: 10px; }
.text-content {
    font-size: 1.05em; line-height: 1.6; color: #333;
    white-space: pre-wrap; word-wrap: break-word;
}
.original-line { font-weight: normal; color: #333; }
.translated-line { font-weight: bold; color: #6A5ACD; }
.history { margin-top: 40px; text-align: left; }
.history h2 { color: #4a90e2; font-size: 1.2em; margin-bottom: 15px; }
.history-list {
    background-color: #f9f9f9; border: 1px solid #eee; border-radius: 8px;
    max-height: 200px; overflow-y: auto; padding: 15px;
}
.history-item { margin-bottom: 15px; padding-bottom: 15px; border-bottom: 1px dashed #eee; }
.history-item:last-child { border-bottom: none; margin-bottom: 0; padding-bottom: 0; }
.history-item p { margin: 5px 0; line-height: 1.5; }
.history-item .original { font-weight: bold; color: #555; }
.history-item .translated { color: #333; }
.btn {
    background-color: #7b4ae2; color: #fff; border: none; border-radius: 5px;
    padding: 10px 20px; margin-top: 20px; cursor: pointer; font-size: 1em;
    transition: background-color 0.2s ease-in-out;
}
.btn:hover { background-color: #6a3ac2; }
footer { margin-top: 40px; font-size: 0.9em; color: #888; }
@media (max-width: 480px) {
    .container { padding: 20px; margin: 10px; }
    .record-btn { width: 70px; height: 70px; }
    .record-btn svg { width: 35px; height: 35px; }
    .status { font-size: 1em; }
    .result-box h2 { font-size: 1.1em; }
    .text-content { font-size: 0.95em; }
    .btn { padding: 8px 15px; font-size: 0.9em; }
}
"""
app_js_content = """
// =================== 最終防彈版 v2 的 app.js (增加自動重置機制) ===================
class RealtimeTranslator {
    constructor() {
        this.recordBtn = document.getElementById("record-btn");
        this.statusDisplay = document.getElementById("status");
        this.translationOutput = document.getElementById("translation-output");
        this.historyList = document.getElementById("history-list");
        this.clearHistoryBtn = document.getElementById("clear-history-btn");
        this.mediaRecorder = null;
        this.audioChunks = [];
        this.isRecording = false;
        this.isProcessing = false;
        this.autoRecordingActive = false;
        this.lastTranslatedTexts = [];

        // --- 新增：自動重置機制的屬性 ---
        this.silentStreakCounter = 0; // 連續靜音計數器
        this.MAX_SILENT_STREAK = 2;   // 連續2次 (12秒) 觸發重置

        this.initEventListeners();
        this.loadHistory();
        this.statusDisplay.textContent = "點擊按鈕開始自動錄音";
    }

    initEventListeners() {
        this.recordBtn.addEventListener("click", () => this.toggleAutoRecording());
        this.clearHistoryBtn.addEventListener("click", () => this.clearHistory());
    }

    toggleAutoRecording() {
        if (this.autoRecordingActive) {
            this.stopAutoRecording();
        } else {
            this.startAutoRecording();
        }
    }

    startAutoRecording() {
        if (this.autoRecordingActive) return;
        this.autoRecordingActive = true;
        this.recordBtn.classList.add("recording");
        this.statusDisplay.textContent = "自動錄音已啟動";
        this.silentStreakCounter = 0; // 每次重新啟動時重置計數器
        this.scheduleNextRecording();
    }

    stopAutoRecording() {
        if (!this.autoRecordingActive) return;
        this.autoRecordingActive = false;
        this.recordBtn.classList.remove("recording");
        this.statusDisplay.textContent = "點擊按鈕開始自動錄音";
        if (this.mediaRecorder && this.isRecording) {
            this.mediaRecorder.stop();
        }
    }

    scheduleNextRecording() {
        if (!this.autoRecordingActive) return;
        if (this.isProcessing) {
            setTimeout(() => this.scheduleNextRecording(), 500);
            return;
        }
        this.statusDisplay.textContent = "準備錄音...";
        setTimeout(() => {
            if (this.autoRecordingActive) {
                this.startRecording();
            }
        }, 500);
    }

    async startRecording() {
        if (this.isRecording) return;
        this.isRecording = true;
        this.audioChunks = [];
        this.statusDisplay.textContent = "錄音中...";
        try {
            const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
            this.mediaRecorder = new MediaRecorder(stream);
            this.mediaRecorder.ondataavailable = event => {
                this.audioChunks.push(event.data);
            };
            this.mediaRecorder.onstop = async () => {
                this.isRecording = false;
                stream.getTracks().forEach(track => track.stop());
                this.isProcessing = true;
                this.statusDisplay.textContent = "處理中...";
                const audioBlob = new Blob(this.audioChunks, { type: "audio/wav" });
                await this.sendAudioForTranslation(audioBlob);
                this.isProcessing = false;
                this.scheduleNextRecording();
            };
            this.mediaRecorder.start();
            setTimeout(() => {
                if (this.mediaRecorder && this.mediaRecorder.state === "recording") {
                    this.mediaRecorder.stop();
                }
            }, 6000);
        } catch (err) {
            console.error("無法獲取麥克風權限:", err);
            this.statusDisplay.textContent = "錯誤: 無法獲取麥克風權限";
            this.isRecording = false;
            this.isProcessing = false;
            this.stopAutoRecording();
        }
    }

    handleSilentDetection() {
        this.silentStreakCounter++;
        console.log(`連續靜音/無效音訊次數: ${this.silentStreakCounter}`);
        if (this.silentStreakCounter >= this.MAX_SILENT_STREAK) {
            console.warn("偵測到連續無效輸入，正在自動重置上下文...");
            this.statusDisplay.textContent = "偵測到連續靜音，已自動重置。";
            this.lastTranslatedTexts = []; // 清除被污染的上下文
            this.silentStreakCounter = 0; // 重置計數器
        } else {
            this.statusDisplay.textContent = "未錄到有效聲音";
        }
    }

    async sendAudioForTranslation(audioBlob) {
        if (audioBlob.size < 5000) {
            console.log(`音訊檔案過小 (${audioBlob.size} bytes)，已攔截。`);
            this.handleSilentDetection(); // 調用靜音處理邏輯
            return;
        }

        try {
            const reader = new FileReader();
            reader.readAsDataURL(audioBlob);
            reader.onloadend = async () => {
                const base64Audio = reader.result.split(',')[1];
                this.statusDisplay.textContent = "正在翻譯...";
                const response = await fetch(API_CONFIG.buildUrl("/api/translate"), {
                    method: "POST",
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ audio: base64Audio, context: this.lastTranslatedTexts.join(" ") })
                });
                const data = await response.json();
                if (response.ok && data.success) {
                    const originalText = data.original_text;
                    const translatedText = data.translated_text;

                    if (translatedText) {
                        // --- 成功收到翻譯，重置計數器 ---
                        this.silentStreakCounter = 0;

                        const html = `
                            <p class="original-line">${originalText}</p>
                            <p class="translated-line">${translatedText}</p>
                        `;
                        this.translationOutput.insertAdjacentHTML("afterbegin", html);
                        this.saveHistory(originalText, translatedText);
                        this.lastTranslatedTexts.push(originalText);
                        if (this.lastTranslatedTexts.length > 2) {
                            this.lastTranslatedTexts.shift();
                        }
                        this.statusDisplay.textContent = "翻譯完成！";
                    } else {
                        // 後端返回了空翻譯，也視為一次無效輸入
                        this.handleSilentDetection();
                    }
                } else {
                    this.statusDisplay.textContent = `翻譯失敗: ${data.error || "未知錯誤"}`;
                    console.error("翻譯 API 錯誤:", data.error);
                }
            };
        } catch (err) {
            this.statusDisplay.textContent = "錯誤: 發送音頻失敗";
            console.error("發送音頻失敗:", err);
        }
    }

    saveHistory(original, translated) {
        let history = JSON.parse(localStorage.getItem("translationHistory")) || [];
        history.unshift({ original, translated, timestamp: new Date().toLocaleString() });
        if (history.length > 10) {
            history = history.slice(0, 10);
        }
        localStorage.setItem("translationHistory", JSON.stringify(history));
        this.renderHistory();
    }

    loadHistory() {
        this.renderHistory();
    }

    renderHistory() {
        const history = JSON.parse(localStorage.getItem("translationHistory")) || [];
        this.historyList.innerHTML = "";
        if (history.length === 0) {
            this.historyList.innerHTML = "<p>沒有歷史記錄。</p>";
            return;
        }
        history.forEach(item => {
            const div = document.createElement("div");
            div.classList.add("history-item");
            div.innerHTML = `
                <p class="original"><strong>日文:</strong> ${item.original}</p>
                <p class="translated"><strong>中文:</strong> ${item.translated}</p>
                <small>${item.timestamp}</small>
            `;
            this.historyList.appendChild(div);
        });
    }

    clearHistory() {
        localStorage.removeItem("translationHistory");
        this.renderHistory();
    }
}

document.addEventListener("DOMContentLoaded", () => {
    new RealtimeTranslator();
});
"""
config_js_content = """
const API_CONFIG = {
    API_BASE_URL: "",
    buildUrl: function(path) { return this.API_BASE_URL + path; }
};
window.API_CONFIG = API_CONFIG;
"""
manifest_json_content = """
{
  "name": "即時日中翻譯", "short_name": "日中翻譯", "start_url": "/",
  "display": "standalone", "background_color": "#ffffff", "theme_color": "#4a90e2",
  "icons": [
    {"src": "/static/icon-192.png", "sizes": "192x192", "type": "image/png"},
    {"src": "/static/icon-512.png", "sizes": "512x512", "type": "image/png"}
  ]
}
"""
sw_js_content = """
const CACHE_NAME = 'realtime-translator-cache-v1';
const urlsToCache = ['/', '/static/styles.css', '/static/app.js', '/static/config.js', '/static/icon-192.png', '/static/icon-512.png'];
self.addEventListener('install', event => {
  event.waitUntil(caches.open(CACHE_NAME).then(cache => cache.addAll(urlsToCache)));
});
self.addEventListener('fetch', event => {
  event.respondWith(caches.match(event.request).then(response => response || fetch(event.request)));
});
"""

with open(os.path.join(frontend_dir, "index.html"), "w", encoding="utf-8") as f: f.write(index_html_content)
with open(os.path.join(frontend_dir, "styles.css"), "w", encoding="utf-8") as f: f.write(styles_css_content)
with open(os.path.join(frontend_dir, "app.js"), "w", encoding="utf-8") as f: f.write(app_js_content)
with open(os.path.join(frontend_dir, "config.js"), "w", encoding="utf-8") as f: f.write(config_js_content)
with open(os.path.join(frontend_dir, "manifest.json"), "w", encoding="utf-8") as f: f.write(manifest_json_content)
with open(os.path.join(frontend_dir, "sw.js"), "w", encoding="utf-8") as f: f.write(sw_js_content)
print("✅ [3/5] 前端應用程式建立完成。")
print("-" * 50)


# STEP 4: 建立並運行 Flask 後端服務
print("🔄 [4/5] 正在設定後端 API 服務...")
app = Flask(__name__, static_folder=frontend_dir, static_url_path='/static')
CORS(app)

@app.route("/")
def index():
    return send_from_directory(app.static_folder, "index.html")

@app.route("/api/health")
def health_check():
    return jsonify({"status": "healthy"})

# 建立一個常見幻覺詞彙的黑名單
HALLUCINATION_BLACKLIST = [
    "謝謝觀看",
    "ご視聴ありがとうございました",
    "最後まで視聴してくださって 本当にありがとうございます",
    "謝謝你的觀看",
    "謝謝收聽",
    "謝謝你的收聽",
    "謝謝你的收聽觀看",
    "感谢您观看",
    "感谢您的观看",
    "感谢收听",
    "感谢您的收听",
    "Thanks for watching",
    "Thank you for watching",
    "Thanks for listening",
    "Thank you for listening",
]

@app.route("/api/translate", methods=["POST"])
def translate_audio():
    if whisper_model is None: # Changed from model to whisper_model
        return jsonify({"success": False, "error": "語音辨識模型未能成功加載，無法提供服務。"}), 500

    data = request.get_json()
    if "audio" not in data:
        return jsonify({"success": False, "error": "No audio data received"}), 400

    temp_audio_path = None
    try:
        audio_data = base64.b64decode(data["audio"])
        audio_segment = AudioSegment.from_file(io.BytesIO(audio_data))

        # --- 新增後端檢查：判斷音訊時長 ---
        # 如果音訊長度小於 0.5 秒，也直接判定為無效
        if len(audio_segment) < 500: # pydub 以毫秒為單位
            print(f"⚠️ 偵測到音訊時長過短 ({len(audio_segment)}ms)，已過濾。")
            return jsonify({"success": True, "original_text": "[ 未偵測到有效語音 ]", "translated_text": ""})

        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as temp_audio_file:
            audio_segment.export(temp_audio_file.name, format="wav")
            temp_audio_path = temp_audio_file.name

        context_prompt = data.get("context", "")
        initial_prompt = '以下是日語對話。' + context_prompt

        segments, info = whisper_model.transcribe( # Changed from model to whisper_model
            temp_audio_path,
            language="ja",
            beam_size=5,
            initial_prompt=initial_prompt,
            log_prob_threshold=-1.0,
            no_speech_threshold=0.6
        )

        original_text = "".join(segment.text for segment in segments).strip()

        # --- 幻覺過濾器 ---
        if not original_text or any(hallucination in original_text for hallucination in HALLUCINATION_BLACKLIST):
            print(f"⚠️ 偵測到幻覺或無語音，已過濾: '{original_text}'")
            return jsonify({"success": True, "original_text": "[ 未偵測到有效語音 ]", "translated_text": ""})

        # Use Gemini for translation
        if gemini_model is None:
             return jsonify({"success": False, "error": "翻譯模型未能成功加載，無法提供服務。"}), 500

        response = gemini_model.generate_content(f"Please translate this Japanese text into Traditional Chinese, and only return the translated text, without including the original text or any additional explanations. Translate the original context details as much as possible, without omitting or condensing the original context:\n\n{original_text}")
        translated_text = response.text.strip()


        return jsonify({"success": True, "original_text": original_text, "translated_text": translated_text})

    except Exception as e:
        print(f"❌ 處理音頻時發生錯誤: {e}")
        return jsonify({"success": False, "error": f"處理音頻時發生錯誤: {str(e)}", "details": traceback.format_exc()}), 500
    finally:
        if temp_audio_path and os.path.exists(temp_audio_path):
            os.remove(temp_audio_path)


def run_flask_app():
    # 使用一個不常用的埠，以避免與 Colab 預設服務衝突
    app.run(host="0.0.0.0", port=6050)

print("✅ [4/5] 後端 API 服務設定完成。")
print("-" * 50)


# STEP 5: 啟動主程序
print("🔄 [5/5] 正在啟動主服務...")
try:
    ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
    ngrok.set_auth_token(ngrok_token)
    print("  ✅ 已從 Colab Secrets 成功加載 NGROK_AUTH_TOKEN。")
except Exception as e:
    print("  ❌ 錯誤: 找不到或無法讀取 Secret 'NGROK_AUTH_TOKEN'。請檢查左側 🔑 分頁中的設定。")

try:
    gemini_api_key = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=gemini_api_key)
    # 使用 gemini-1.5-flash，這是目前最適合此應用的高效模型
    gemini_model = genai.GenerativeModel('gemini-2.0-flash')
    print("  ✅ 已從 Colab Secrets 成功加載 GEMINI_API_KEY 並初始化 Gemini 模型。")
except Exception as e:
    print("  ❌ 錯誤: 找不到或無法使用 Secret 'GEMINI_API_KEY'。請檢查左側 🔑 分頁中的設定。")
    gemini_model = None # 確保模型為 None，以便 API 端點可以正確回報錯誤

# 只有在 Gemini 模型成功初始化後才啟動服務
if gemini_model:
    flask_thread = threading.Thread(target=run_flask_app)
    flask_thread.daemon = True
    flask_thread.start()
    print("  - 後端服務已在背景執行緒中啟動。")
    time.sleep(3)

    try:
        public_url = ngrok.connect(6050)
        print("✅ [5/5] 服務啟動成功！")
        print("=" * 50)
        print(f"   🌐 公開 URL: {public_url}")
        print(f"   📱 請在瀏覽器中開啟此 URL 開始使用")
        print("=" * 50)
    except Exception as e:
        print(f"  ❌ 錯誤: 無法啟動 ngrok 通道。請檢查您的 NGROK_AUTH_TOKEN 是否正確。詳細錯誤: {e}")
else:
    print("❌ [5/5] 由於 Gemini 模型初始化失敗，服務未啟動。")

# 保持主執行緒運行，以防 Colab 閒置超時
# 如果您想停止服務，請手動中斷執行階段
try:
    while True:
        time.sleep(3600) # 每小時喚醒一次
except KeyboardInterrupt:
    print("\n🛑 服務已手動停止")
finally:
    ngrok.kill()
    print("   - ngrok tunnel 已關閉。")

🔄 [1/5] 正在安裝所有依賴項...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
✅ [1/5] 所有依賴項安裝完成。
--------------------------------------------------
🔄 [2/5] 正在加載 AI 模型...
  - 正在加載 Faster-Whisper 模型 (medium)...
  ✅ Whisper 模型已成功加載到 CUDA。
  - Gemini 模型將在稍後初始化...
✅ [2/5] AI 模型準備就緒。
--------------------------------------------------
🔄 [3/5] 正在建立前端應用程式...
  ⚠️ 警告: 找不到圖示檔案。應用程式仍可運行，但可能無法正確顯示圖示或作為 PWA 安裝。
✅ [3/5] 前端應用程式建立完成。
--------------------------------------------------
🔄 [4/5] 正在設定後端 API 服務...
✅ [4/5] 後端 API 服務設定完成。
--------------------------------------------------
🔄 [5/5] 正在啟動主服務...
  ✅ 已從 Colab Secrets 成功加載 NGROK_AUTH_TOKEN。
  ✅ 已從 Colab Secrets 成功加載 GEMINI_API_KEY 並初始化 Gemini 模型。
  - 後端服務已在背景執行緒中啟動。
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 6050 is in use by another program. Either identify and stop that program, or start the server with a different port.


✅ [5/5] 服務啟動成功！
   🌐 公開 URL: NgrokTunnel: "https://99e35445cde4.ngrok-free.app" -> "http://localhost:6050"
   📱 請在瀏覽器中開啟此 URL 開始使用


INFO:werkzeug:127.0.0.1 - - [18/Jul/2025 01:30:49] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jul/2025 01:30:50] "GET /static/styles.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jul/2025 01:30:50] "GET /static/config.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jul/2025 01:30:50] "GET /static/app.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jul/2025 01:30:51] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/Jul/2025 01:30:54] "GET /static/styles.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jul/2025 01:31:09] "POST /api/translate HTTP/1.1" 200 -


⚠️ 偵測到幻覺或無語音，已過濾: '字幕提供者のみなさん、ご視聴ありがとうございました。'


...

⚠️ 偵測到幻覺或無語音，已過濾: '字幕提供者のみなさん、ご視聴ありがとうございました。'


INFO:werkzeug:127.0.0.1 - - [18/Jul/2025 01:34:11] "POST /api/translate HTTP/1.1" 200 -


⚠️ 偵測到幻覺或無語音，已過濾: '字幕提供者の皆さん、ご視聴ありがとうございました。'


INFO:werkzeug:127.0.0.1 - - [18/Jul/2025 01:34:14] "GET / HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jul/2025 01:34:14] "GET /static/styles.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jul/2025 01:34:14] "GET /static/config.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [18/Jul/2025 01:34:14] "GET /static/app.js HTTP/1.1" 304 -



🛑 服務已手動停止
   - ngrok tunnel 已關閉。
